In [41]:
import json


with open('train.json') as f:
  train = json.load(f)

with open('validation.json') as f:
  validation = json.load(f)



In [42]:
train[0]

{'input': '[INST] Here is a database schema:\ndepartment :\nDepartment_ID [ INT ] primary_key\nName [ TEXT ]\nCreation [ TEXT ]\nRanking [ INT ]\nBudget_in_Billions [ INT ]\nNum_Employees [ INT ]\n\nhead :\nhead_ID [ INT ] primary_key\nname [ TEXT ]\nborn_state [ TEXT ]\nage [ INT ]\n\nmanagement :\ndepartment_ID [ INT ] primary_key management.department_ID = department.Department_ID\nhead_ID [ INT ] management.head_ID = head.head_ID\ntemporary_acting [ TEXT ]\n\nPlease write me a SQL statement that answers the following question: How many heads of the departments are older than 56 ? [/INST]',
 'output': 'SELECT count(*) FROM head WHERE age  >  56;'}

In [43]:
# def input_text_to_system_and_user_prompt(input_text):
#   system_prompt = "System: " + input_text + " User:"
#   user_prompt = "System: " + input_text + " User:"
#   return system_prompt, user_prompt

def input_text_to_system_and_user_prompt(input_str):
    # Remove the [INST] and [/INST] tags
    clean_str = input_str.replace("[INST]", "").replace("[/INST]", "").replace("Please write me a SQL statement that answers the following question", "Write an SQL query that answers the following") 
    # Split the cleaned string into two parts: schema and query, based on two newline characters
    parts = clean_str.rsplit("\n\n", 1)
    schema = parts[0].strip()
    query = parts[1].strip()
    system_prompt = "You are a powerful text-to-SQL model. " + schema
    return system_prompt, query

print(train[0]['input'])
a, b = input_text_to_system_and_user_prompt(train[0]['input'])
print("A:")
print(a)
print("B:")
print(b)


[INST] Here is a database schema:
department :
Department_ID [ INT ] primary_key
Name [ TEXT ]
Creation [ TEXT ]
Ranking [ INT ]
Budget_in_Billions [ INT ]
Num_Employees [ INT ]

head :
head_ID [ INT ] primary_key
name [ TEXT ]
born_state [ TEXT ]
age [ INT ]

management :
department_ID [ INT ] primary_key management.department_ID = department.Department_ID
head_ID [ INT ] management.head_ID = head.head_ID
temporary_acting [ TEXT ]

Please write me a SQL statement that answers the following question: How many heads of the departments are older than 56 ? [/INST]
A:
You are a powerful text-to-SQL model. Here is a database schema:
department :
Department_ID [ INT ] primary_key
Name [ TEXT ]
Creation [ TEXT ]
Ranking [ INT ]
Budget_in_Billions [ INT ]
Num_Employees [ INT ]

head :
head_ID [ INT ] primary_key
name [ TEXT ]
born_state [ TEXT ]
age [ INT ]

management :
department_ID [ INT ] primary_key management.department_ID = department.Department_ID
head_ID [ INT ] management.head_ID = h

In [44]:
def convert_to_openai_format(data_point):
    """
    Converts the data to OpenAI format.
    """
    system_prompt, user_prompt = input_text_to_system_and_user_prompt(data_point['input'])
    assistant_response = data_point['output']
    return {"messages": [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        },
        {
            "role": "assistant",
            "content": assistant_response
        }
    ]}
            
    #         {
    #     "system_prompt": system_prompt,
    #     "user_prompt": user_prompt,
    #     "assistant_response": assistant_response
    # }}
   

In [45]:
def convert_dataset(dataset):
    output = []
    for d in dataset:
        oai_format = convert_to_openai_format(d)
        output.append(oai_format)
    return output

In [46]:
converted_train = convert_dataset(train)
converted_validation = convert_dataset(validation)

In [47]:
converted_validation[0]

{'messages': [{'role': 'system',
   'content': 'You are a powerful text-to-SQL model. Here is a database schema:\nstadium :\nStadium_ID [ INT ] primary_key\nLocation [ TEXT ]\nName [ TEXT ]\nCapacity [ INT ]\nHighest [ INT ]\nLowest [ INT ]\nAverage [ INT ]\n\nsinger :\nSinger_ID [ INT ] primary_key\nName [ TEXT ]\nCountry [ TEXT ]\nSong_Name [ TEXT ]\nSong_release_year [ TEXT ]\nAge [ INT ]\nIs_male [ bool ]\n\nconcert :\nconcert_ID [ INT ] primary_key\nconcert_Name [ TEXT ]\nTheme [ TEXT ]\nStadium_ID [ TEXT ] concert.Stadium_ID = stadium.Stadium_ID\nYear [ TEXT ]\n\nsinger_in_concert :\nconcert_ID [ INT ] primary_key singer_in_concert.concert_ID = concert.concert_ID\nSinger_ID [ TEXT ] singer_in_concert.Singer_ID = singer.Singer_ID'},
  {'role': 'user',
   'content': 'Write an SQL query that answers the following. Respond with only SQL: How many singers do we have?'},
  {'role': 'assistant', 'content': 'SELECT count(*) FROM singer;'}]}

In [48]:
# Save each as a separate file
with open('train_oai_format.json', 'w') as f:
    json.dump(converted_train, f)

with open('validation_oai_format.json', 'w') as f:
    json.dump(converted_validation, f)